# Import Library

In [1]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import sys
from collections import Counter
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load Data

In [2]:
df = pd.read_csv('../assets/SplitData/2021_10_26.csv')
df

,title,summary,link,published
0,EEC-รฟท.ยันยังไม่โอนสิทธิ “แอร์พอร์ตลิงก์” จนก...,EEC-รฟท.ยันยังไม่โอนสิทธิ “แอร์พอร์ตลิงก์” จนก...,https://thaipublica.org/2021/10/high-speed-tra...,2021_10_26
1,วิสัยทัศน์ “3 เหลี่ยมเศรษฐกิจ” ของลาว,ผู้นำ 3 ประเทศเห็นชอบการกำหนดเป้าหมายแผนแม่บทค...,https://thaipublica.org/2021/10/pundop52/,2021_10_26
2,บจ.ไทยเดินหน้าขับเคลื่อนธุรกิจยั่งยืน คะแนน CG...,บจ. ไทย ทำคะแนนประเมิน CG สูงสุดรอบ 21 ปี และเ...,https://thaipublica.org/2021/10/%e0%b8%9a%e0%b...,2021_10_26
3,“กวาร์ตาราโร่” แซงระห่ำ มิซาโน่ ผงาดครองบัลลัง...,ฟาบิโอ กวาร์ตาราโร่ #20 ดาวบิดเฟรนช์แมน สังกัด...,https://www.prachachat.net/motoring/news-788814,2021_10_26
4,"ศบค.พบป่วยโควิดวันนี้ (26 ต.ค.) 7,706 ราย ATK ...",ศบค.พบผู้ติดเชื้อโควิดรายใหม่วันนี้ (26 ต.ค.) ...,https://www.prachachat.net/general/news-788744,2021_10_26
...,...,...,...,...
827,"โควิดวันนี้ รวมรักษาตัวใน รพ. 98,150 ราย ติดเช...",ศบค. รายงานสถานการณ์โควิดวันนี้ 26 ต.ค. 64 มีผ...,https://www.thairath.co.th/news/politic/2227921,2021_10_26
828,รมช.สธ.ขออย่าตื่นตระหนก หลังพบคนไทย ติด &quot;...,รมช.สธ. ขอประชาชน อย่าตื่นตระหนกเกินไป ย้ำ โปร...,https://www.thairath.co.th/news/politic/2227961,2021_10_26
829,สธ. ย้ำ ไทยยังไม่พบ เดลตาพลัส สายพันธุ์ AY.4.2...,"""นพ.ศุภกิจ"" ยืนยัน ไทยยังไม่พบ เดลตาพลัส สายพั...",https://www.thairath.co.th/news/local/2227966,2021_10_26
830,Squid Game สะท้อนวิกฤติหนี้เกาหลีใต้ เมื่อ ปชช...,ซีรีส์ สควิดเกม ดังเป็นพลุแตกไปทั่วโลก นอกจากจ...,https://www.thairath.co.th/news/foreign/2227630,2021_10_26


# Read Stop-Word Method

In [3]:
def read_stop_word(filename):
    text_file = open(f"../assets/StopWordList/{filename}.txt", "r", encoding='utf-8')
    content = text_file.read()
    dict_list = content.split("\n")
    text_file.close()
    return dict_list

In [4]:
newmm_stopword = read_stop_word("newmm_stopword")

In [5]:
attacut_stopword = read_stop_word("attacut_stopword")

# Word Break Method

In [6]:
NEWMM = "newmm"
ATTACUT = "attacut"
list_column = df.columns.values.tolist()
column_name = list_column[1]

def word_break(news,engine):
    segment = tokenize(str(news),keep_whitespace=False,engine=engine)
    return segment

In [7]:
atta_list = []
newmm_list = []

def wordBreak(df):
    l = len(df)
    for i in tqdm(range(l)):
        wb_list = word_break(df.iloc[i][column_name],ATTACUT)
        remove_sw = [word for word in wb_list  if word not in attacut_stopword]
        atta_list.append(remove_sw)
        wb_list = word_break(df.iloc[i][column_name],NEWMM)
        remove_sw = [word for word in wb_list  if word not in newmm_stopword]
        newmm_list.append(remove_sw)

wordBreak(df)

100%|██████████| 832/832 [00:32<00:00, 25.61it/s]


# LSC Method

In [8]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    lcs_atta = []
    lcs_newmm = []
    for i in tqdm(range(l-1)):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.extend(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.extend(lcs_newmm_list)

    lcs_atta_counter = Counter(lcs_atta)
    lcs_newmm_counter = Counter(lcs_newmm)
    return lcs_atta_counter, lcs_newmm_counter

In [9]:
def LCS_result(data):
    # print("result")
    df_result = pd.DataFrame(data.items(), columns=['LCS', 'round'])
    df_result['total_round'] = df_result['round'].sum()
    df_result.sort_values(['round'], ascending=False, inplace=True, ignore_index=True)
    return df_result

# Result

In [10]:
lcs_atta, lcs_newmm = LCS(atta_list, newmm_list)

100%|██████████| 831/831 [00:02<00:00, 327.52it/s]


In [11]:
result_newmm = LCS_result(lcs_newmm)
result_newmm[:20]

,LCS,round,total_round
0,ขวัญอุษามณี,13,315
1,“พรพิมล,13,315
2,เดลตาพลัสAY,13,315
3,รทวอลล์มอเตอร์,12,315
4,ฝุ่นละอองPM2.5,11,315
5,ตึกภักดีบดินทร์ทำเนียบรัฐบาล,7,315
6,พลัสAY4.2,6,315
7,เจ้าหญิงโกะราชวงศ์,6,315
8,ร.อ.นัสพรหมเผ่า,6,315
9,เช้านี้แกว่งไซด์เวย์,6,315


In [12]:
len(result_newmm)

164

In [13]:
trend = result_newmm[result_newmm["LCS"].str.contains("แอนชิลี")]
trend

,LCS,round,total_round


In [14]:
news = df[df["summary"].str.contains("แอนชิลี")]
news

,title,summary,link,published


In [15]:
result_atta = LCS_result(lcs_atta)
result_atta.head(20)

,LCS,round,total_round
0,ตึกภักดีบดินทร์ทำเนียบรัฐบาล,8,111
1,นายธนกร วังบุญคงชนะสำนักนายกรัฐมนตรีกลาโหม,5,111
2,นิคมอุตสาหกรรมนวนคร,3,111
3,ประธานาธิบดีโจ ไบเดนสุดยอดสมาคมประชาชาติ,3,111
4,เลื่อนอุทธรณ์วิสามัญชัยภูมิ,2,111
5,สายพันธุ์เดลตาพลัส,2,111
6,ช้างเอฟเอคัพ,2,111
7,ชัยภูมิป่าแสกองทัพบก,2,111
8,250รูปพรรณ636,2,111
9,นายสันติพร้อมพัฒน์รมช.นายอนุชา นาคาศัยสำนักนาย...,2,111
